In [19]:
%pip install pyodbc prophet matplotlib pandas requests


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install prophet


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install pyodbc


Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import pyodbc
from prophet import Prophet
import matplotlib.pyplot as plt
import requests
import os


server = '41.207.70.18,60001'   # include port with comma
database = 'brits_ncba'
username = 'brits_ncba'
password = '6fuAFMCeQvtk7L8wn5sl'

connection_string = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

try:
    conn = pyodbc.connect(connection_string)
    print("✅ Connected to SQL Server successfully!")
except Exception as e:
    print("❌ Database connection failed:", e)
    raise

✅ Connected to SQL Server successfully!


In [25]:
# 🧩 Cell 3 — Fetch Data with Terminal Info
query = """
SELECT 
    t.terminalid,
    t.name AS terminal_name,
    t.location,
    tr.trandate,
    tr.cashin
FROM [transactions] AS tr
JOIN [terminals] AS t
    ON tr.terminalid = t.terminalid
WHERE 
    tr.trandate >= DATEADD(DAY, -180, GETDATE())
    AND tr.cashin IS NOT NULL
"""

conn = pyodbc.connect(connection_string)
df = pd.read_sql(query, conn)
conn.close()

# Clean
df['trandate'] = pd.to_datetime(df['trandate'])
df['cashin'] = pd.to_numeric(df['cashin'], errors='coerce')

terminals = df['terminalid'].unique()
print(f"✅ Retrieved data for {len(terminals)} terminals (joined with terminals info)")
df.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12268\3402276194.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Retrieved data for 15 terminals (joined with terminals info)


,terminalid,terminal_name,location,trandate,cashin
0,CDMARTC006,ART CAFFE WESTLANDS SQUARE,WESTLANDS SQUARE,2025-10-28 00:19:09.610,2300.0
1,CDMARTC006,ART CAFFE WESTLANDS SQUARE,WESTLANDS SQUARE,2025-10-28 00:22:11.363,44250.0
2,CDMGAG001,GASANDGO MOMBASA ROAD,MOMBASA ROAD,2025-10-28 02:04:00.867,51500.0
3,CDMARTC007,ART CAFFE NAROK CDM,RUBIS MARA JUNCTION,2025-10-28 06:54:45.653,29250.0
4,CDMGAG001,GASANDGO MOMBASA ROAD,MOMBASA ROAD,2025-10-28 07:00:43.293,129000.0


In [26]:
output_dir = "forecasts"
os.makedirs(output_dir, exist_ok=True)
all_forecasts = []

for terminal in terminals:
    terminal_data = df[df['terminalid'] == terminal].dropna(subset=['cashin'])
    
    if len(terminal_data) < 3:
        print(f"⚠️ Skipping {terminal} (not enough data)")
        continue

    print(f"\n🔮 Forecasting cashin for terminal: {terminal}")

    data = terminal_data.rename(columns={'trandate': 'ds', 'cashin': 'y'})
    model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=False)
    model.fit(data)

    # Predict next 30 days
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    forecast['terminalid'] = terminal

    # Add terminal name and location if available
    terminal_info = terminal_data[['terminal_name', 'location']].iloc[0].to_dict()
    forecast['terminal_name'] = terminal_info.get('terminal_name', '')
    forecast['location'] = terminal_info.get('location', '')

    # Save chart
    plt.figure(figsize=(10, 5))
    model.plot(forecast)
    plt.title(f"Cashin Forecast - {terminal}")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{terminal}_cashin_forecast.png")
    plt.close()

    simplified = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'terminalid', 'terminal_name', 'location']]
    all_forecasts.append(simplified)



🔮 Forecasting cashin for terminal: CDMARTC006


16:09:58 - cmdstanpy - INFO - Chain [1] start processing
16:10:00 - cmdstanpy - INFO - Chain [1] done processing



🔮 Forecasting cashin for terminal: CDMGAG001


16:10:02 - cmdstanpy - INFO - Chain [1] start processing
16:10:02 - cmdstanpy - INFO - Chain [1] done processing



🔮 Forecasting cashin for terminal: CDMARTC007


...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [27]:
import pandas as pd
import plotly.express as px

# Combine all forecasts into one DataFrame
all_forecasts_df = pd.concat(all_forecasts, ignore_index=True)

# Example: visualize forecast for a single terminal
terminal_to_plot = all_forecasts_df['terminalid'].unique()[0]  # change if you want another
plot_df = all_forecasts_df[all_forecasts_df['terminalid'] == terminal_to_plot]

fig = px.line(
    plot_df,
    x='ds',
    y=['yhat', 'yhat_lower', 'yhat_upper'],
    title=f"📈 Cash-in Forecast for Terminal {terminal_to_plot}",
    labels={'ds': 'Date', 'value': 'Forecast (KES)'},
)
fig.show()


In [28]:
# 🧩 Cell 5 — Combine Forecasts
if all_forecasts:
    all_forecasts_df = pd.concat(all_forecasts)
    csv_path = f"{output_dir}/cashin_forecasts.csv"
    all_forecasts_df.to_csv(csv_path, index=False)
    print(f"📂 Saved combined forecast to: {csv_path}")
else:
    print("⚠️ No forecasts generated.")


📂 Saved combined forecast to: forecasts/cashin_forecasts.csv


In [29]:
import requests

try:
    backend_url = "http://localhost:5000/api/cashForecasts"  # change port if needed
    print(f"🚀 Sending forecast data to backend: {backend_url}")

    # Convert datetime columns (like 'ds') to string (ISO format)
    all_forecasts_df['ds'] = all_forecasts_df['ds'].astype(str)

    # Convert Decimal or float64 columns just in case
    all_forecasts_df = all_forecasts_df.astype({
        'yhat': 'float',
        'yhat_lower': 'float',
        'yhat_upper': 'float'
    })

    payload = all_forecasts_df.to_dict(orient='records')
    response = requests.post(backend_url, json=payload)

    if response.status_code == 200:
        print("✅ Forecast data sent successfully to backend!")
    else:
        print(f"❌ Failed to send data — {response.status_code}: {response.text}")

except Exception as e:
    print(f"⚠️ Error sending data to backend: {e}")


🚀 Sending forecast data to backend: http://localhost:5000/api/cashForecasts
❌ Failed to send data — 500: Database insert error
